In [3]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [4]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

# 加载研究区

In [5]:
## 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
## 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
## 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
## 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
## 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)


# 加载自动选取的样本点和人工选取的样本点

## 加载人工选取的样本点

In [ ]:
### 人工选取的样本点
# 要求：水体类别属性为 label; 影像属性为 Image_id 。
man_samples = ee.FeatureCollection('users/311605001111/TIANJIN_2020')
print(man_samples.size().getInfo())

## 加载自动选取的样本点

In [ ]:
### 自动选取的样本点
# 要求：水体类别属性为 waterclass; 影像属性为 Image_id 。
auto_samples = ee.FeatureCollection('users/311605001111/tianjin_2020')
# auto_samples = ee.FeatureCollection('users/311605001111/PearlRiverDelta_2018')
print(auto_samples.size().getInfo())

# 获取人工点和自动点都采集的影像ID

In [ ]:
manual_samples = man_samples.distinct('Image_id')
id = ee.List(manual_samples.aggregate_array('Image_id'))
print(id.size().getInfo())
print(id.getInfo())


automatic_samples = auto_samples.distinct('Image_id')
id = ee.List(automatic_samples.aggregate_array('Image_id'))
print(id.size().getInfo())
print(id.getInfo())

In [ ]:
filter = ee.Filter.equals(**{'leftField': 'Image_id','rightField': 'Image_id'})
id_col = ee.List(ee.Join.simple().apply(automatic_samples, manual_samples, filter).aggregate_array('Image_id'))
print(id_col.size().getInfo())
print(id_col.getInfo())

# 在同一景影像上，人工点与自动点的验证函数

In [ ]:
# 验证函数
water_image = ee.Image.constant(1).toFloat().rename('label')
land_image = ee.Image.constant(0).toFloat().rename('label')
def verify(img_id):
    auto_point = auto_samples.filter(ee.Filter.eq('Image_id',img_id))
    man_point = man_samples.filter(ee.Filter.eq('Image_id',img_id))
    water = water_image.clipToCollection(man_point.filter(ee.Filter.eq('label',1)))
    land = land_image.clipToCollection(man_point.filter(ee.Filter.eq('label',0)))
    image_waterclass = ee.ImageCollection([water,land]).sum()
    inter_points = image_waterclass.sampleRegions(**{
        'collection': auto_point,
        'properties': ['waterclass'],
        'scale': 30,
        'geometries': True,
    })
    return inter_points

# 验证

In [ ]:
# com_SamplePoints = ee.FeatureCollection(id_col.map(verify)).flatten()
# print(com_SamplePoints.aggregate_array('label').getInfo())
# print(com_SamplePoints.size().getInfo())

In [ ]:
# 利用误差矩阵进行验证
com_SamplePoints = ee.FeatureCollection(id_col.map(verify)).flatten()
test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total caccuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

# 核心代码

In [7]:
# 核心代码

###数据
# 人工选取的样本点
man_samples = ee.FeatureCollection('users/311605001111/WUHAN_2000merge')
print('manual-sample number:{}'.format(man_samples.size().getInfo()))
# 自动选取的样本点
samples = ee.FeatureCollection('users/311605001111/wuhan_2000')
print('auto-sample number:{}'.format(samples.size().getInfo()))
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample
auto_samples = k_mean(samples)
print("筛选后，剩余的样本数： ",auto_samples.size().getInfo())



### 唯一ID
manual_samples = man_samples.distinct('Image_id')
automatic_samples = auto_samples.distinct('Image_id')
filter = ee.Filter.equals(**{'leftField': 'Image_id','rightField': 'Image_id'})
id_col = ee.List(ee.Join.simple().apply(automatic_samples, manual_samples, filter).aggregate_array('Image_id'))

### 验证函数
water_image = ee.Image.constant(1).toFloat().rename('label')
land_image = ee.Image.constant(0).toFloat().rename('label')
def verify(img_id):
    auto_point = auto_samples.filter(ee.Filter.eq('Image_id',img_id))
    man_point = man_samples.filter(ee.Filter.eq('Image_id',img_id))
    water = water_image.clipToCollection(man_point.filter(ee.Filter.eq('label',1)))
    land = land_image.clipToCollection(man_point.filter(ee.Filter.eq('label',0)))
    image_waterclass = ee.ImageCollection([water,land]).sum()
    inter_points = image_waterclass.sampleRegions(**{
        'collection': auto_point,
        'properties': ['waterclass','Image_id'],
        'scale': 30,
        'geometries': True,
    })
    return inter_points

### 精度验证
SamplePoints = ee.FeatureCollection(id_col.map(verify)).flatten()
water = SamplePoints.filter(ee.Filter.eq('label',1)).randomColumn('random').sort('random').limit(100)
no_water = SamplePoints.filter(ee.Filter.eq('label',0)).randomColumn('random').sort('random').limit(400)
com_SamplePoints = ee.FeatureCollection([water,no_water]).flatten()

test_accuracy = SamplePoints.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

manual-sample number:1983
auto-sample number:3771
筛选后，剩余的样本数：  3699
[[955, 1], [0, 178]]
total accuracy:0.9991181657848325
kappa:0.9966755981871165
[[400, 0], [0, 100]]
total accuracy:1
kappa:1


In [8]:
# 利用误差矩阵进行验证
diff_water = SamplePoints.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',0))
diff_nowater = SamplePoints.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
cor_water = SamplePoints.filter(ee.Filter.eq('label',1)).filter(ee.Filter.eq('waterclass',1)).randomColumn('random').sort('random').limit(100-diff_water.size().getInfo())
cor_nowater = SamplePoints.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',0)).randomColumn('random').sort('random').limit(400-diff_nowater.size().getInfo())
total_samples = ee.FeatureCollection([diff_water,diff_nowater,cor_water,cor_nowater]).flatten()

test_accuracy = total_samples.errorMatrix('label', 'waterclass')
print(test_accuracy.getInfo())
print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

[[399, 1], [0, 100]]
total accuracy:0.998
kappa:0.9937733499377335


In [ ]:
a = SamplePoints.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1))
id = SamplePoints.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1)).distinct('Image_id').aggregate_array('Image_id')

for i in range(0,id.size().getInfo(),1):
    print(id.get(i).getInfo())
    print('total:',a.filter(ee.Filter.eq('Image_id',id.get(i))).size().getInfo())

In [ ]:
q = a.filter(ee.Filter.eq('Image_id','LANDSAT/LE07/C01/T1_SR/LE07_123033_20001210'))
print(q.getInfo())

In [ ]:
116.00273148914086, 38.82783660973387


In [ ]:
var b = ee.Geometry.Point([117.00910929827164, 38.890901875718455]);
Map.addLayer(b,{'color': 'red', 'pointSize': 100,'fillColor': 'red'},'b');
Map.centerObject(b,14);

In [ ]:
# LANDSAT/LE07/C01/T1_SR/LE07_122039_20000306
# LANDSAT/LT05/C01/T1_SR/LT05_123039_20000727

a = SamplePoints.filter(ee.Filter.eq('Image_id','LANDSAT/LT05/C01/T1_SR/LT05_119028_20101015')).filter(ee.Filter.eq('label',1))
print(a.size().getInfo())

# shp格式
selector = ['Image_id','label','waterclass']
geemap.ee_export_vector_to_drive(a, description='LT05_119028_20101015_cor', folder='corPoints2', file_format='shp', selectors=selector)

In [ ]:
print('total water:',auto_samples.filter(ee.Filter.eq('waterclass',1)).size().getInfo())
print('total no-water:',auto_samples.filter(ee.Filter.eq('waterclass',0)).size().getInfo())
id = auto_samples.distinct('Image_id').aggregate_array('Image_id')
print(id.getInfo())

# 在各景的分布（选择前的）
for i in range(0,id.size().getInfo(),1):
    print(id.get(i).getInfo())
    print('water:',auto_samples.filter(ee.Filter.eq('Image_id',id.get(i))).filter(ee.Filter.eq('waterclass',1)).size().getInfo())
    print('no-water:',auto_samples.filter(ee.Filter.eq('Image_id',id.get(i))).filter(ee.Filter.eq('waterclass',0)).size().getInfo())

In [ ]:
a = ee.FeatureCollection('users/311605001111/TIANJIN_2020merge')
print('total water:',a.filter(ee.Filter.eq('label',1)).size().getInfo())
print('total no-water:',a.filter(ee.Filter.eq('label',0)).size().getInfo())
id = a.distinct('Image_id').aggregate_array('Image_id')
print(id.getInfo())

# 在各景的分布（选择前的）
for i in range(0,id.size().getInfo(),1):
    print(id.get(i).getInfo())
    print('water:',a.filter(ee.Filter.eq('Image_id',id.get(i))).filter(ee.Filter.eq('label',1)).size().getInfo())
    print('no-water:',a.filter(ee.Filter.eq('Image_id',id.get(i))).filter(ee.Filter.eq('label',0)).size().getInfo())

In [ ]:
# 在各景的分布（选择后的）
for i in range(0,id_col.size().getInfo(),1):
    print(com_SamplePoints.filter(ee.Filter.eq('Image_id',id_col.get(i))).size().getInfo())
    print(id_col.get(i).getInfo())
    print('water:',com_SamplePoints.filter(ee.Filter.eq('Image_id',id_col.get(i))).filter(ee.Filter.eq('label',1)).size().getInfo())
    print('no-water:',com_SamplePoints.filter(ee.Filter.eq('Image_id',id_col.get(i))).filter(ee.Filter.eq('label',0)).size().getInfo())

In [ ]:
# 在各景的分布（选择前的

for i in range(0,id_col.size().getInfo(),1):
    print(ee.FeatureCollection(id_col.map(verify).get(i)).size().getInfo())
    print(ee.FeatureCollection(id_col.map(verify).get(i)).distinct('Image_id').aggregate_array('Image_id').getInfo())
    print('water:',ee.FeatureCollection(id_col.map(verify).get(i)).filter(ee.Filter.eq('label',1)).size().getInfo())
    print('no-water:',ee.FeatureCollection(id_col.map(verify).get(i)).filter(ee.Filter.eq('label',0)).size().getInfo())

# 验证点，空间分布

In [ ]:
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
def maskSR(img):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = img.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    azimuth = img.get('SOLAR_AZIMUTH_ANGLE')
    zenith = img.get('SOLAR_ZENITH_ANGLE')
    image = img.lt(0)
    bands = image.select('B2').add(image.select('B3')).add(image.select('B4')).add(image.select('B5')).add(image.select('B6')).add(image.select('B7'))
    outlier = bands.gt(0).remap([0,1],[1,0]).rename('outlier')
    return img.updateMask(mask).updateMask(ee.Terrain.hillShadow(elevation,azimuth,zenith,200,True)).updateMask(outlier)

# 图像可视化参数
visParams = {'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4}

l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
       .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa']) \
       .filterBounds(roi) \
       .filter(ee.Filter.lt('CLOUD_COVER',30)) \
       .filterDate('2020-01-01', '2020-12-31')          
Map.addLayer(l8.map(maskSR).mosaic().clip(roi), visParams,'image mosaic')
print(l8.size().getInfo())

In [ ]:
# q = com_SamplePoints.draw('red')
# Map.addLayer(q, {}, 'q')

refer_img = l8.map(maskSR).median().clip(roi).visualize(**{'bands': ['B5', 'B6', 'B4'],'min': 0,'max': 3000,'gamma': 1.4,'opacity':0.65})
blend = refer_img.blend(com_SamplePoints.filter(ee.Filter.eq('label',0)).draw('red')).blend(com_SamplePoints.filter(ee.Filter.eq('label',1)).draw('yellow'))
Map.addLayer(blend, {}, "Blend")

In [ ]:
# 合成影像
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
# # 大庆市
# region = [123.9355, 45.8362,125.3055, 46.8762]
# # 天津市
# region = [115.9244, 38.6272,117.4944,39.6372]
# # 武汉市
# region = [113.7393, 29.8642,115.0993, 30.9242]
# # 珠三角
# region = [112.7614, 22.2347,114.0514, 23.2547]
# 青藏高原
region = [89.8826, 34.6579,91.3626, 35.8279]
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(blend, region=region)
cartoee.add_gridlines(ax, interval=[0.3,0.2], linestyle=":")
ax.set_title(label = 'Region E', fontsize=28)

In [ ]:
samples = ee.FeatureCollection('users/311605001111/PearlRiverDelta_2015')
print('sample number:{}'.format(samples.size().getInfo()))

# 水体的样本点数
samplepoint_water = samples.filter(ee.Filter.eq('waterclass',1))
print('water sample number:{}'.format(samplepoint_water.size().getInfo()))

# 非水体的样本点数
samplepoint_land = samples.filter(ee.Filter.eq('waterclass',0))
print('nowater sample number:{}'.format(samplepoint_land.size().getInfo()))

## 循环

In [ ]:
# cluster与waterclass的对比
def Compare(feature):
    cluster = ee.Number(feature.get('cluster'))
    waterclass = ee.Number(feature.get('waterclass'))
    ft = ee.Algorithms.If(cluster.eq(waterclass),feature.set({'eq':1}),feature.set({'eq':0}))
    return ft
# k-mean聚类
def k_mean(sampleSET):
    clusterer = ee.Clusterer.wekaKMeans(2).train(sampleSET,bands)
    result = sampleSET.cluster(clusterer)
    right = result.map(Compare).filter(ee.Filter.eq('eq',0))
    error = result.map(Compare).filter(ee.Filter.eq('eq',1))
    filtered_sample = ee.FeatureCollection(ee.Algorithms.If(right.size().gt(error.size()),right,error))
    return filtered_sample
def verify(img_id):
    auto_point = auto_samples.filter(ee.Filter.eq('Image_id',img_id))
    man_point = man_samples.filter(ee.Filter.eq('Image_id',img_id))
    water = water_image.clipToCollection(man_point.filter(ee.Filter.eq('label',1)))
    land = land_image.clipToCollection(man_point.filter(ee.Filter.eq('label',0)))
    image_waterclass = ee.ImageCollection([water,land]).sum()
    inter_points = image_waterclass.sampleRegions(**{
        'collection': auto_point,
        'properties': ['waterclass','Image_id'],
        'scale': 30,
        'geometries': True,
    })
    return inter_points


for i in ['2000','2005','2010','2015','2020']:
    print("{} year".format(i))
    id_man = 'users/311605001111/DAQING_' + i
    id_auto = 'users/311605001111/daqing_' + i
    man_samples = ee.FeatureCollection(id_man)
    print('manual-sample number:{}'.format(man_samples.size().getInfo()))
    samples = ee.FeatureCollection(id_auto)
    print('auto-sample number:{}'.format(samples.size().getInfo()))
    bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','mNDWI','CWI','AWEI','EWI','EVI']
    auto_samples = k_mean(samples)
    print("筛选后，剩余的样本数： ",auto_samples.size().getInfo())
    ### 唯一ID
    manual_samples = man_samples.distinct('Image_id')
    automatic_samples = auto_samples.distinct('Image_id')
    filter = ee.Filter.equals(**{'leftField': 'Image_id','rightField': 'Image_id'})
    id_col = ee.List(ee.Join.simple().apply(automatic_samples, manual_samples, filter).aggregate_array('Image_id'))
    ### 验证函数
    water_image = ee.Image.constant(1).toFloat().rename('label')
    land_image = ee.Image.constant(0).toFloat().rename('label')
    ### 精度验证
    com_SamplePoints = ee.FeatureCollection(id_col.map(verify)).flatten()
    print("consistent sample number:",com_SamplePoints.size().getInfo())
    test_accuracy = com_SamplePoints.errorMatrix('label', 'waterclass')
    print(test_accuracy.getInfo())
    print('total accuracy:{}'.format(test_accuracy.accuracy().getInfo()))
    print('kappa:{}'.format(test_accuracy.kappa().getInfo()))

In [ ]:
miss = com_SamplePoints.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.eq('Image_id','LANDSAT/LT05/C01/T1_SR/LT05_122039_20051107'))
print(miss.size().getInfo())
print(miss.distinct('Image_id').getInfo())

In [ ]:
miss = com_SamplePoints.filter(ee.Filter.eq('label',0)).filter(ee.Filter.eq('waterclass',1)).filter(ee.Filter.eq('Image_id','LANDSAT/LT05/C01/T1_SR/LT05_122039_20051107'))
print(miss.size().getInfo())
print(miss.distinct('Image_id').aggregate_array('Image_id').getInfo())

In [ ]:
LANDSAT/LE07/C01/T1_SR/LE07_122044_20201124
LANDSAT/LE07/C01/T1_SR/LE07_122044_20201124
LANDSAT/LE07/C01/T1_SR/LE07_122044_20200921
LANDSAT/LE07/C01/T1_SR/LE07_122044_20200921
LANDSAT/LE07/C01/T1_SR/LE07_122044_20200921